In [ ]:
import tensorflow as tf
import numpy as np
import os
import cv2
from skimage import morphology

def preprocess_and_skeletonize_alternative(image):
    
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    binary = gray > 127
    
    skeleton = morphology.skeletonize(binary)
    
    skeleton = (skeleton * 255).astype(np.uint8)
    
    skeleton_rgb = cv2.cvtColor(skeleton, cv2.COLOR_GRAY2RGB)
    
    skeleton_rgb = skeleton_rgb.astype(np.float32)
    preprocessed = tf.keras.applications.mobilenet_v2.preprocess_input(skeleton_rgb)
    
    return preprocessed

def create_model():
    IMG_SHAPE = (224, 224, 3)
    
    base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                   include_top=False,
                                                   weights=None)
    base_model.trainable = False
    
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(62, activation='softmax')
    ])
    
    return model

def evaluate_all_folders():
    print("Starting baseline model evaluation...")
    
    model = create_model()
    model.load_weights('baseline.weights.h5') 
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=preprocess_and_skeletonize_alternative
    )
    
    data_root = "."
    user_folders = [d for d in os.listdir(data_root) 
                   if os.path.isdir(os.path.join(data_root, d)) and d.endswith('_results_final')]
    
    print(f"Found {len(user_folders)} user folders: {[d.replace('_results_final', '') for d in user_folders]}")
    
    accuracies = []
    losses = []
    results_data = []
    
    for user_folder in user_folders:
        user_initials = user_folder.replace('_results_final', '')
        test_path = os.path.join(data_root, user_folder, f"{user_initials}_inverted")
        
        if not os.path.exists(test_path):
            print(f"Skipping {user_initials}: Path {test_path} not found")
            continue
        
        print(f"\nEvaluating user: {user_initials}")
        print(f"   Test path: {test_path}")
        
        test_generator = datagen.flow_from_directory(
            test_path,
            target_size=(224, 224),
            batch_size=64,
            shuffle=False,
            class_mode='categorical'
        )
        
        results = model.evaluate(test_generator, verbose=1)
        test_loss = results[0]
        test_accuracy = results[1]
        
        accuracies.append(test_accuracy)
        losses.append(test_loss)
        results_data.append({
            'user': user_initials,
            'accuracy': test_accuracy,
            'loss': test_loss,
            'samples': test_generator.samples
        })
        
        print(f"{user_initials}: Accuracy={test_accuracy:.4f}, Loss={test_loss:.4f}, Samples={test_generator.samples}")
    
    if accuracies:
        mean_accuracy = np.mean(accuracies)
        std_accuracy = np.std(accuracies)
        mean_loss = np.mean(losses)
        std_loss = np.std(losses)
        
        print("\n" + "="*50)
        print(" BASELINE MODEL RESULTS SUMMARY")
        print(f"Mean Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
        print(f"Mean Loss: {mean_loss:.4f} ± {std_loss:.4f}")
        print(f"Total Users Evaluated: {len(accuracies)}")
        print(f"Best User Accuracy: {max(accuracies):.4f}")
        print(f"Worst User Accuracy: {min(accuracies):.4f}")
        
        print("\nIndividual Results:")
        for result in results_data:
            print(f"   {result['user']}: {result['accuracy']:.4f} ({result['samples']} samples)")
        
   
        
        return {
            'mean_accuracy': mean_accuracy,
            'std_accuracy': std_accuracy,
            'mean_loss': mean_loss,
            'std_loss': std_loss,
            'individual_results': results_data
        }
    else:
        print("No user folders evaluated successfully!")
        return None

if __name__ == "__main__":
    results = evaluate_all_folders()



Starting baseline model evaluation...


Found 19 user folders: ['7MK', 'AB3', 'AN7', 'AS5', 'BC', 'BP', 'DDJ', 'DTT', 'HJ', 'HM', 'ID', 'JK1', 'JKK', 'KP4', 'MJ8', 'MMK', 'MZ', 'SA', 'TSH']

Evaluating user: 7MK
   Test path: .\7MK_results_final\7MK_inverted
Found 1939 images belonging to 62 classes.


31/31 [==============================] - 50s 2s/step - loss: 2.2907 - accuracy: 0.4776
7MK: Accuracy=0.4776, Loss=2.2907, Samples=1939

Evaluating user: AB3
   Test path: .\AB3_results_final\AB3_inverted
Found 2167 images belonging to 62 classes.
34/34 [==============================] - 41s 1s/step - loss: 2.0934 - accuracy: 0.4887
AB3: Accuracy=0.4887, Loss=2.0934, Samples=2167

Evaluating user: AN7
   Test path: .\AN7_results_final\AN7_inverted
Found 1884 images belonging to 62 classes.
30/30 [==============================] - 35s 1s/step - loss: 2.1299 - accuracy: 0.4936
AN7: Accuracy=0.4936, Loss=2.1299, Samples=1884

Evaluating user: AS5
   Test path: .\AS5_results_final\AS5_invert